<a href="https://colab.research.google.com/github/shahabas9/Deep_learning_basics/blob/main/ANN_hyper_parameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
data=pd.read_csv("diabetes.csv")

In [ ]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
x=data.iloc[:,:-1]

In [ ]:
y=data["Outcome"]

In [ ]:
x.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [ ]:
x=x.values

In [ ]:
y=y.values

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


In [ ]:
scaler=StandardScaler()

In [ ]:
x=scaler.fit_transform(x)

In [ ]:
x

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=43)

In [ ]:
Layers=[
    tf.keras.layers.Dense(32,activation="relu",input_dim=8),
    tf.keras.layers.Dense(1,activation="sigmoid")

]
model=tf.keras.models.Sequential(Layers)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                288       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(x_train,y_train,batch_size=32,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
20/20 [==============================] - 6s 20ms/step - loss: 0.6275 - accuracy: 0.6645 - val_loss: 0.6092 - val_accuracy: 0.6948
Epoch 2/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5877 - accuracy: 0.6906 - val_loss: 0.5800 - val_accuracy: 0.7208
Epoch 3/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5595 - accuracy: 0.7117 - val_loss: 0.5609 - val_accuracy: 0.7273
Epoch 4/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5379 - accuracy: 0.7280 - val_loss: 0.5481 - val_accuracy: 0.7403
Epoch 5/10
20/20 [==============================] - 0s 6ms/step - loss: 0.5212 - accuracy: 0.7296 - val_loss: 0.5381 - val_accuracy: 0.7532
Epoch 6/10
20/20 [==============================] - 0s 7ms/step - loss: 0.5086 - accuracy: 0.7362 - val_loss: 0.5308 - val_accuracy: 0.7597
Epoch 7/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4975 - accuracy: 0.7459 - val_loss: 0.5253 - val_accuracy: 0.7597
Epoch 8/10
20/20 [=

In [ ]:
!pip install -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.6 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

In [ ]:
def build_model(hp):
  Layers=[
    tf.keras.layers.Dense(32,activation="relu",input_dim=8),
    tf.keras.layers.Dense(1,activation="sigmoid")

  ]
  model=tf.keras.models.Sequential(Layers)
  Optimizer=hp.Choice("optimizer",values=["adam","sgd","adadelta","rmsprop"])
  model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer=Optimizer)
  return model

In [ ]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5)

In [ ]:
tuner.search(x_train,y_train,epochs=6,validation_data=(x_test,y_test))

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7532467246055603
Total elapsed time: 00h 00m 10s


In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [35]:
model=tuner.get_best_models(num_models=1)[0]

In [36]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [37]:
model.fit(x_train,y_train,batch_size=32,validation_data=(x_test,y_test),epochs=10,initial_epoch=6)

Epoch 7/10
20/20 [==============================] - 1s 12ms/step - loss: 0.5045 - accuracy: 0.7557 - val_loss: 0.5139 - val_accuracy: 0.7532
Epoch 8/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4959 - accuracy: 0.7573 - val_loss: 0.5097 - val_accuracy: 0.7532
Epoch 9/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4883 - accuracy: 0.7590 - val_loss: 0.5049 - val_accuracy: 0.7597
Epoch 10/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4815 - accuracy: 0.7655 - val_loss: 0.5023 - val_accuracy: 0.7662


In [38]:
def build_model(hp):
  units=hp.Int("units",min_value=8,max_value=128,step=8)
  Layers=[
    tf.keras.layers.Dense(units=units,activation="relu",input_dim=8),
    tf.keras.layers.Dense(1,activation="sigmoid")

  ]
  model=tf.keras.models.Sequential(Layers)
  Optimizer="rmsprop"
  model.compile(loss="binary_crossentropy",metrics=["accuracy"],optimizer=Optimizer)
  return model

In [39]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="mydir")

In [40]:
tuner.search(x_train,y_train,epochs=6,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 11s


In [41]:
tuner.get_best_hyperparameters()[0].values

{'units': 120}

In [42]:
model=tuner.get_best_models(num_models=1)[0]

In [43]:
model.fit(x_train,y_train,batch_size=32,validation_data=(x_test,y_test),epochs=10,initial_epoch=6)

Epoch 7/10
20/20 [==============================] - 1s 17ms/step - loss: 0.4644 - accuracy: 0.7818 - val_loss: 0.5013 - val_accuracy: 0.7792
Epoch 8/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4578 - accuracy: 0.7801 - val_loss: 0.5018 - val_accuracy: 0.7727
Epoch 9/10
20/20 [==============================] - 0s 7ms/step - loss: 0.4530 - accuracy: 0.7818 - val_loss: 0.5020 - val_accuracy: 0.7662
Epoch 10/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4491 - accuracy: 0.7866 - val_loss: 0.5028 - val_accuracy: 0.7727


In [48]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=8))

    for i in range(hp.Int("num_layers", min_value=1, max_value=10)):
        model.add(tf.keras.layers.Dense(120, activation="relu"))

    model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

    optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)  # Set the learning rate as needed
    model.compile(loss="binary_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    return model

In [49]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="mydir1")

In [50]:
tuner.search(x_train,y_train,epochs=6,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 20s


In [51]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1}

In [52]:
model=tuner.get_best_models(num_models=1)[0]

In [53]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 120)               1080      
                                                                 
 dense_1 (Dense)             (None, 120)               14520     
                                                                 
 dense_2 (Dense)             (None, 1)                 121       
                                                                 
Total params: 15,721
Trainable params: 15,721
Non-trainable params: 0
_________________________________________________________________


In [54]:
import tensorflow as tf
import keras
from keras.layers import Dense
from keras.models import Sequential

In [55]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value= 1,max_value=10)):
    if counter == 0:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh']),
                      input_dim=8))


    else:
      model.add(Dense(hp.Int('units' + str(i),min_value= 8,max_value=128, step=8),
                      activation=hp.Choice('activation' + str(i), values=['relu', 'tanh'])))

    counter += 1


  model.add(Dense(1, activation='sigmoid'))


  LOSS_FUNCTION = "binary_crossentropy"
  OPTIMIZER = hp.Choice('optimizer', values = ['adam', 'sgd', 'rmsprop', 'adadelta'])
  METRICS = ["accuracy"]

  model.compile(loss=LOSS_FUNCTION,
                optimizer=OPTIMIZER,
                metrics=METRICS)

  return model


In [56]:
tuner=kt.RandomSearch(build_model,objective="val_accuracy",max_trials=5,directory="mydir2")

In [57]:
tuner.search(x_train,y_train,epochs=6,validation_data=(x_test,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.7857142686843872

Best val_accuracy So Far: 0.7857142686843872
Total elapsed time: 00h 00m 24s


In [58]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 4,
 'units0': 72,
 'activation0': 'tanh',
 'optimizer': 'rmsprop',
 'units1': 16,
 'activation1': 'tanh',
 'units2': 120,
 'activation2': 'relu',
 'units3': 128,
 'activation3': 'tanh',
 'units4': 56,
 'activation4': 'relu',
 'units5': 40,
 'activation5': 'relu',
 'units6': 48,
 'activation6': 'relu',
 'units7': 88,
 'activation7': 'tanh',
 'units8': 112,
 'activation8': 'tanh',
 'units9': 40,
 'activation9': 'relu'}

In [59]:
model=tuner.get_best_models(num_models=1)[0]

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 72)                648       
                                                                 
 dense_1 (Dense)             (None, 16)                1168      
                                                                 
 dense_2 (Dense)             (None, 120)               2040      
                                                                 
 dense_3 (Dense)             (None, 128)               15488     
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 19,473
Trainable params: 19,473
Non-trainable params: 0
_________________________________________________________________


In [61]:
model.fit(x_train,y_train,batch_size=32,validation_data=(x_test,y_test),epochs=10,initial_epoch=6)

Epoch 7/10
20/20 [==============================] - 2s 14ms/step - loss: 0.4583 - accuracy: 0.7883 - val_loss: 0.4962 - val_accuracy: 0.7792
Epoch 8/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4597 - accuracy: 0.7752 - val_loss: 0.4883 - val_accuracy: 0.7662
Epoch 9/10
20/20 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.7932 - val_loss: 0.5054 - val_accuracy: 0.7468
Epoch 10/10
20/20 [==============================] - 0s 5ms/step - loss: 0.4488 - accuracy: 0.7850 - val_loss: 0.4997 - val_accuracy: 0.7792
